### Universidad del Valle de Guatemala<br>Data Science<br>Laboratorio6<br>

#### Integrantes:<br>- Christopher García 20541<br>- Andrea Lam 20102

In [1]:
# Se importan librerías
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image
import torchvision.datasets as dset
import torchvision.transforms as transforms
import numpy as np

### Se accede a Kaggle para obtener datos

In [2]:
!pip install kaggle

In [3]:
import os

# Obtener el directorio actual donde se encuentra el archivo de Jupyter Notebook
current_directory = os.path.dirname(os.path.abspath('__file__'))

# Ruta completa al directorio .kaggle en el directorio actual
kaggle_directory = os.path.join(current_directory, ".kaggle")

# Crear el directorio .kaggle si no existe
if not os.path.exists(kaggle_directory):
    os.mkdir(kaggle_directory)


In [4]:
!copy .kaggle\kaggle.json %userprofile%\.kaggle\

        1 file(s) copied.


In [5]:
! chmod 600 ~/.kaggle/kaggle.json

'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!kaggle datasets download -d jessicali9530/celeba-dataset

celeba-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
import os
import zipfile


# Obtener la ubicación actual de trabajo
current_directory = os.getcwd()

# Definir una ubicación para la extracción
extract_path = os.path.join(current_directory, "celeba-dataset")

# Crear la carpeta de extracción si no existe
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Extraer el archivo ZIP
zip_ref = zipfile.ZipFile('celeba-dataset.zip', 'r')
zip_ref.extractall(extract_path)
zip_ref.close()


In [23]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensiona las imágenes a 128x128 píxeles
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [24]:
dataset = dset.ImageFolder(
    root="./celeba-dataset/img_align_celeba",
    transform=transform  # Utiliza la transformación modificada
)


## Preparacion de datos

In [25]:
# Define las variables
ngpu = 1
ngf = 64
nc = 3
nz = 100
lr = 0.0002
beta1 = 0.5
batch_size = 128
num_epochs = 5
workers = 2

In [26]:
# Especifica la ruta al directorio de datos
data_dir = 'celeba-dataset/img_align_celeba/'

# Define la transformación para preprocesar las imágenes
transform = transforms.Compose([
    transforms.Resize(128),            # Redimensiona las imágenes a 128x128 píxeles
    transforms.CenterCrop(128),        # Recorta las imágenes al centro
    transforms.ToTensor(),             # Convierte las imágenes en tensores
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normaliza los valores de los píxeles a [-1, 1]
])

# Carga el conjunto de datos
dataset = dset.ImageFolder(root="celeba-dataset/img_align_celeba", transform=transform)

# Crea un DataLoader para facilitar el acceso a los datos en lotes
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

# Define el dispositivo (CPU o GPU)
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


## Implementacion de la GAN

In [27]:
# Define el generador y el discriminador
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Capa de entrada: Z dimension -> ngf*8 dimension
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            # Capa 2: ngf*8 dimension -> ngf*4 dimension
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            # Capa 3: ngf*4 dimension -> ngf*2 dimension
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            # Capa 4: ngf*2 dimension -> ngf dimension
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            # Capa de salida: ngf dimension -> nc canales (3 canales en RGB)
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)


In [28]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Capa de entrada: 3 canales de color
            nn.Conv2d(nc, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Capa de salida: un solo canal
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


**# Definicion perdida y optimizadores**

In [29]:
# Define el generador y el discriminador
generator = Generator().to(device)
discriminator = Discriminator().to(device)


# Define la función de pérdida para la GAN
criterion = nn.BCELoss()

# Define los optimizadores para el generador y el discriminador
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))



## Entrenamiento de la GAN

In [33]:
real_labels = torch.ones((batch_size, 1, 1, 1), device=device)
fake_labels = torch.zeros((batch_size, 1, 1, 1), device=device)

In [34]:
# Entrenamiento de la GAN
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        real_images, _ = data  # Obtén un lote de imágenes reales y sus etiquetas (si es necesario)

        # Inicializa los gradientes del discriminador y el generador
        optimizer_D.zero_grad()

        # Entrena al discriminador con imágenes reales
        output = discriminator(real_images.to(device))
        loss_D_real = criterion(output, real_labels)
        loss_D_real.backward()

        # Genera un lote de imágenes falsas
        fake_images = generator(torch.randn(batch_size, nz, 1, 1, device=device))

        # Entrena al discriminador con imágenes falsas
        output = discriminator(fake_images.detach())
        loss_D_fake = criterion(output, fake_labels)
        loss_D_fake.backward()

        # Pérdida total del discriminador
        loss_D = loss_D_real + loss_D_fake

        # Actualiza los pesos del discriminador
        optimizer_D.step()

        # Inicializa los gradientes del generador
        optimizer_G.zero_grad()

        # Entrena al generador para engañar al discriminador
        output = discriminator(fake_images)
        loss_G = criterion(output, real_labels)
        loss_G.backward()

        # Actualiza los pesos del generador
        optimizer_G.step()

        # Imprime estadísticas durante el entrenamiento (puedes personalizar esto)
        if i % 100 == 0:
            print(f'Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(dataloader)}] Loss_D: {loss_D.item()}, Loss_G: {loss_G.item()}')

        # Guarda imágenes generadas por el generador al final de cada época (opcional)
        if (i + 1) == len(dataloader):
            with torch.no_grad():
                fake = generator(torch.randn(64, nz, 1, 1, device=device)).detach().cpu()
            save_image(fake, f"images/epoch_{epoch}.png", normalize=True)

ValueError: Using a target size (torch.Size([128, 1, 1, 1])) that is different to the input size (torch.Size([128, 1, 5, 5])) is deprecated. Please ensure they have the same size.